In [ ]:
#!kill -9 -1

In [ ]:
#!echo 3 | sudo tee /proc/sys/vm/drop_caches

In [ ]:
from google.colab import drive
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, apriori, association_rules

In [ ]:
drive.mount('/content/drive')

# "/content/drive/MyDrive/Colab Notebooks/data/Instacart/"
aisles = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/Instacart/aisles.csv')
departments = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/Instacart/departments.csv')
order_products_prior = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/Instacart/order_products__prior.csv')
order_products_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/Instacart/order_products__train.csv')
orders = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/Instacart/orders.csv')
products = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/Instacart/products.csv')

Mounted at /content/drive


In [ ]:
print(aisles.head())
print(aisles.shape)

   aisle_id                       aisle
0         1       prepared soups salads
1         2           specialty cheeses
2         3         energy granola bars
3         4               instant foods
4         5  marinades meat preparation
(134, 2)


In [ ]:
print(departments.head(21))
print(departments.shape)

    department_id       department
0               1           frozen
1               2            other
2               3           bakery
3               4          produce
4               5          alcohol
5               6    international
6               7        beverages
7               8             pets
8               9  dry goods pasta
9              10             bulk
10             11    personal care
11             12     meat seafood
12             13           pantry
13             14        breakfast
14             15     canned goods
15             16       dairy eggs
16             17        household
17             18           babies
18             19           snacks
19             20             deli
20             21          missing
(21, 2)


In [ ]:
print(order_products_prior.head())
print(order_products_prior.shape)

   order_id  product_id  add_to_cart_order  reordered
0         2       33120                  1          1
1         2       28985                  2          1
2         2        9327                  3          0
3         2       45918                  4          1
4         2       30035                  5          0
(32434489, 4)


In [ ]:
print(order_products_train.head())
print(order_products_train.shape)

   order_id  product_id  add_to_cart_order  reordered
0         1       49302                  1          1
1         1       11109                  2          1
2         1       10246                  3          0
3         1       49683                  4          0
4         1       43633                  5          1
(1384617, 4)


In [ ]:
print(orders.head())
print(orders.shape)

   order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
0   2539329        1    prior             1          2                  8   
1   2398795        1    prior             2          3                  7   
2    473747        1    prior             3          3                 12   
3   2254736        1    prior             4          4                  7   
4    431534        1    prior             5          4                 15   

   days_since_prior_order  
0                     NaN  
1                    15.0  
2                    21.0  
3                    29.0  
4                    28.0  
(3421083, 7)


In [ ]:
print(products.head())
print(products.shape)

   product_id                                       product_name  aisle_id  \
0           1                         Chocolate Sandwich Cookies        61   
1           2                                   All-Seasons Salt       104   
2           3               Robust Golden Unsweetened Oolong Tea        94   
3           4  Smart Ones Classic Favorites Mini Rigatoni Wit...        38   
4           5                          Green Chile Anytime Sauce         5   

   department_id  
0             19  
1             13  
2              7  
3              1  
4             13  
(49688, 4)


In [ ]:
# Create product list for each order
merged = order_products_prior.merge(products[['product_id', 'product_name', 'department_id']], on='product_id', how='left')
merged.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,department_id
0,2,33120,1,1,Organic Egg Whites,16
1,2,28985,2,1,Michigan Organic Kale,4
2,2,9327,3,0,Garlic Powder,13
3,2,45918,4,1,Coconut Butter,13
4,2,30035,5,0,Natural Sweetener,13


In [ ]:
print(f"Number of rows before subsetting: {merged.shape[0]}")

Number of rows before subsetting: 32434489


In [ ]:
# Department name: frozen, bakery, dry goods pasta, meat seafood, pantry, breakfast, canned goods, dairy eggs
# Department id: 1, 3, 9, 12, 13, 14, 15, 16

merged = merged[merged['department_id'].isin([1, 3, 9, 12, 13, 14, 15, 16])]
merged.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,department_id
0,2,33120,1,1,Organic Egg Whites,16
2,2,9327,3,0,Garlic Powder,13
3,2,45918,4,1,Coconut Butter,13
4,2,30035,5,0,Natural Sweetener,13
6,2,40141,7,1,Original Unflavored Gelatine Mix,13


In [ ]:
merged.groupby('department_id')['department_id'].count()

,department_id
department_id,
1,2236432
3,1176787
9,866627
12,708931
13,1875577
14,709569
15,1068058
16,5414016


In [ ]:
# Select top 1000 saled products
top_1000_products = merged['product_name'].value_counts().head(1000).index

merged = merged[merged['product_name'].isin(top_1000_products)]

In [ ]:
print(f"shape: {merged.shape}")
print(f"unique products: {merged['product_name'].nunique()}")

shape: (8354964, 6)
unique products: 1000


In [ ]:
basket = merged.groupby('order_id')['product_name'].apply(list).tolist()
print(f"length of the basket: {len(basket)}")

length of the basket: 2484264


TransactionEncoder

In [ ]:
te = TransactionEncoder()
te_ary = te.fit(basket).transform(basket)
df = pd.DataFrame(te_ary, columns=te.columns_)

In [ ]:
print(f"df shape: {df.shape}")

df shape: (2484264, 1000)


In [ ]:
df.head()

,0% Fat Free Organic Milk,0% Greek Strained Yogurt,1% Low Fat Milk,1% Lowfat Milk,100% Grated Parmesan Cheese,100% Lactose Free Fat Free Milk,100% Lactose Free Reduced Fat Calcium Enriched Milk,100% Pure Pumpkin,100% Recycled Aluminum Foil,100% Whole Wheat Bread,...,YoKids Squeeze Organic Blueberry Blue Yogurt,YoKids Squeeze! Organic Strawberry Flavor Yogurt,"YoKids Squeezers Organic Low-Fat Yogurt, Strawberry",YoKids Strawberry Banana/Strawberry Yogurt,Yobaby Organic Plain Yogurt,Yoghurt Blueberry,"Yogurt, Lowfat, Strawberry","Yogurt, Strained Low-Fat, Coconut",Yotoddler Organic Pear Spinach Mango Yogurt,with Crispy Almonds Cereal
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
print(df.sum(axis=1).head(10))

0    2
1    4
2    1
3    5
4    1
5    5
6    3
7    2
8    1
9    2
dtype: int64


In [ ]:
print(df.sum(axis=0).sort_values(ascending=False).head(10))

Organic Whole Milk                   137905
Organic Half & Half                   76360
Half & Half                           69217
100% Whole Wheat Bread                60816
Organic Whole String Cheese           59676
Organic Unsweetened Almond Milk       57895
Blueberries                           55946
Boneless Skinless Chicken Breasts     50281
Extra Virgin Olive Oil                50255
Unsweetened Almondmilk                49569
dtype: int64


In [ ]:
# Only Top 300 products
top_300_products = df.sum().sort_values(ascending=False).head(300).index

df_top_300 = df[top_300_products]

In [ ]:
print(f"df_top_300 shape: {df_top_300.shape}")

df_top_300 shape: (2484264, 300)


In [ ]:
order_sizes = df_top_300.sum(axis=1)

df_top = df_top_300[order_sizes > order_sizes.quantile(0.9)] # also filtered out orders, top 10% only for cost

In [ ]:
print(f"df_top shape: {df_top.shape}")

df_top shape: (245996, 300)


Apriori

In [ ]:
import psutil

# RAM
total_ram = psutil.virtual_memory().total / 1e9
used_ram = psutil.virtual_memory().used / 1e9
available_ram = psutil.virtual_memory().available / 1e9

print(f"Total RAM: {total_ram:.2f} GB")
print(f"Used RAM: {used_ram:.2f} GB")
print(f"Available RAM: {available_ram:.2f} GB")

Total RAM: 54.75 GB
Used RAM: 9.15 GB
Available RAM: 44.96 GB


In [ ]:
#import multiprocessing
#num_cores = multiprocessing.cpu_count()
#print(f"Number of cores: {num_cores}")

In [ ]:
## divide df into num_cores dfs
#df_split = np.array_split(df_top, num_cores)

## for parallel executing and support over 0.3%
#def run_apriori(df_chunk):
#    return apriori(df_chunk, min_support=0.003, use_colnames=True)

#with multiprocessing.Pool(num_cores) as pool:
#    results = pool.map(run_apriori, df_split)

#frequent_itemsets = pd.concat(results)

min_support = 0.01 --> 4 mins \
min_support = 0.009 --> 6 mins

In [ ]:
frequent_itemsets = apriori(df_top, min_support=0.01, use_colnames=True)

In [ ]:
# only combinations with 2 or more items
#frequent_itemsets = frequent_itemsets[frequent_itemsets['itemsets'].apply(lambda x: len(x) > 1)]

In [ ]:
frequent_itemsets.head(10)

,support,itemsets
0,0.157080,(Organic Whole Milk)
1,0.075692,(Organic Half & Half)
2,0.078684,(Half & Half)
3,0.061322,(100% Whole Wheat Bread)
4,0.102916,(Organic Whole String Cheese)
5,0.055371,(Organic Unsweetened Almond Milk)
6,0.043350,(Blueberries)
7,0.060631,(Boneless Skinless Chicken Breasts)
8,0.052745,(Extra Virgin Olive Oil)
9,0.043102,(Unsweetened Almondmilk)


In [ ]:
print(frequent_itemsets.shape)

(274, 2)


In [ ]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

In [ ]:
# Most frequent items
frequent_itemsets.sort_values(by="support", ascending=False)

,support,itemsets
0,0.157080,(Organic Whole Milk)
4,0.102916,(Organic Whole String Cheese)
2,0.078684,(Half & Half)
11,0.077221,(Grated Parmesan)
1,0.075692,(Organic Half & Half)
14,0.062290,(Organic Black Beans)
3,0.061322,(100% Whole Wheat Bread)
7,0.060631,(Boneless Skinless Chicken Breasts)
16,0.060538,(Shredded Parmesan)
32,0.058460,(Total 2% with Strawberry Lowfat Greek Strained Yogurt)


In [ ]:
# association rule
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

In [ ]:
# Display the most relevant columns
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10))

                                                                                                 antecedents  \
0                                                     (Total 2% Lowfat Greek Strained Yogurt With Blueberry)   
1                                                         (Total 2% Lowfat Greek Strained Yogurt with Peach)   
2                                                                               (Vanilla Skyr Nonfat Yogurt)   
3                                                                                 (Non Fat Raspberry Yogurt)   
4                                                                 (Nonfat Icelandic Style Strawberry Yogurt)   
5                                                                      (Non Fat Acai & Mixed Berries Yogurt)   
6  (Total 2% with Strawberry Lowfat Greek Strained Yogurt, Total 2% Lowfat Greek Strained Yogurt with Peach)   
7   (Total 2% Lowfat Greek Strained Yogurt with Peach, Total 2% Lowfat Greek Strained Yogurt With Bluebe

In [ ]:
rules.sort_values(by="lift", ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
5,(Non Fat Acai & Mixed Berries Yogurt),(Icelandic Style Skyr Blueberry Non-fat Yogurt),0.016846,0.037765,0.010512,0.624035,16.524225,1.0,0.009876,2.559373,0.955580,0.238385,0.609279,0.451199
4,(Nonfat Icelandic Style Strawberry Yogurt),(Icelandic Style Skyr Blueberry Non-fat Yogurt),0.022110,0.037765,0.012829,0.580254,15.364919,1.0,0.011994,2.292421,0.956055,0.272704,0.563780,0.459987
3,(Non Fat Raspberry Yogurt),(Icelandic Style Skyr Blueberry Non-fat Yogurt),0.032606,0.037765,0.018817,0.577110,15.281676,1.0,0.017586,2.275380,0.966062,0.365006,0.560513,0.537694
2,(Vanilla Skyr Nonfat Yogurt),(Icelandic Style Skyr Blueberry Non-fat Yogurt),0.033728,0.037765,0.017525,0.519585,13.758442,1.0,0.016251,2.002927,0.959686,0.324721,0.500731,0.491816
6,"(Total 2% with Strawberry Lowfat Greek Strained Yogurt, Total 2% Lowfat Greek Strained Yogurt with Peach)",(Total 2% Lowfat Greek Strained Yogurt With Blueberry),0.022630,0.044651,0.012529,0.553620,12.398779,1.0,0.011518,2.140212,0.940634,0.228822,0.532757,0.417105
7,"(Total 2% Lowfat Greek Strained Yogurt with Peach, Total 2% Lowfat Greek Strained Yogurt With Blueberry)",(Total 2% with Strawberry Lowfat Greek Strained Yogurt),0.018643,0.058460,0.012529,0.672045,11.495756,1.0,0.011439,2.870945,0.930356,0.194020,0.651683,0.443178
0,(Total 2% Lowfat Greek Strained Yogurt With Blueberry),(Total 2% with Strawberry Lowfat Greek Strained Yogurt),0.044651,0.058460,0.025797,0.577749,9.882766,1.0,0.023187,2.229813,0.940823,0.333666,0.551532,0.509513
1,(Total 2% Lowfat Greek Strained Yogurt with Peach),(Total 2% with Strawberry Lowfat Greek Strained Yogurt),0.041988,0.058460,0.022630,0.538968,9.219384,1.0,0.020176,2.042244,0.930608,0.290811,0.510342,0.463038


In [ ]:
rules = rules[rules['antecedents'].apply(lambda x: len(x) > 1) | rules['consequents'].apply(lambda x: len(x) > 1)]

In [ ]:
# Display the most relevant columns
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10))

                                                                                                 antecedents  \
6  (Total 2% with Strawberry Lowfat Greek Strained Yogurt, Total 2% Lowfat Greek Strained Yogurt with Peach)   
7   (Total 2% Lowfat Greek Strained Yogurt with Peach, Total 2% Lowfat Greek Strained Yogurt With Blueberry)   

                                               consequents   support  \
6   (Total 2% Lowfat Greek Strained Yogurt With Blueberry)  0.012529   
7  (Total 2% with Strawberry Lowfat Greek Strained Yogurt)  0.012529   

   confidence       lift  
6    0.553620  12.398779  
7    0.672045  11.495756  


In [ ]:
rules.sort_values(by="lift", ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
6,"(Total 2% with Strawberry Lowfat Greek Strained Yogurt, Total 2% Lowfat Greek Strained Yogurt with Peach)",(Total 2% Lowfat Greek Strained Yogurt With Blueberry),0.022630,0.044651,0.012529,0.553620,12.398779,1.0,0.011518,2.140212,0.940634,0.228822,0.532757,0.417105
7,"(Total 2% Lowfat Greek Strained Yogurt with Peach, Total 2% Lowfat Greek Strained Yogurt With Blueberry)",(Total 2% with Strawberry Lowfat Greek Strained Yogurt),0.018643,0.058460,0.012529,0.672045,11.495756,1.0,0.011439,2.870945,0.930356,0.194020,0.651683,0.443178


In [ ]:
pd.reset_option("display.max_rows")
pd.reset_option("display.max_columns")
pd.reset_option("display.max_colwidth")

In [ ]:
while True:
    pass

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Creating DataFrame with the given data
data = {
    "Antecedents": [
        "Non Fat Acai & Mixed Berries Yogurt",
        "Nonfat Icelandic Style Strawberry Yogurt",
        "Non Fat Raspberry Yogurt",
        "Vanilla Skyr Nonfat Yogurt",
        "Total 2% with Strawberry Lowfat Greek Strained Yogurt, Total 2% Lowfat Greek Strained Yogurt with Peach",
        "Total 2% Lowfat Greek Strained Yogurt with Peach, Total 2% Lowfat Greek Strained Yogurt With Blueberry",
        "Total 2% Lowfat Greek Strained Yogurt With Blueberry",
        "Total 2% Lowfat Greek Strained Yogurt with Peach"
    ],
    "Consequents": [
        "Icelandic Style Skyr Blueberry Non-fat Yogurt",
        "Icelandic Style Skyr Blueberry Non-fat Yogurt",
        "Icelandic Style Skyr Blueberry Non-fat Yogurt",
        "Icelandic Style Skyr Blueberry Non-fat Yogurt",
        "Total 2% Lowfat Greek Strained Yogurt With Blueberry",
        "Total 2% with Strawberry Lowfat Greek Strained Yogurt",
        "Total 2% with Strawberry Lowfat Greek Strained Yogurt",
        "Total 2% with Strawberry Lowfat Greek Strained Yogurt"
    ],
    "Support": [0.010512, 0.012829, 0.018817, 0.017525, 0.012529, 0.012529, 0.025797, 0.022630],
    "Confidence": [0.624035, 0.580254, 0.577110, 0.519585, 0.553620, 0.672045, 0.577749, 0.538968],
    "Lift": [16.524225, 15.364919, 15.281676, 13.758442, 12.398779, 11.495756, 9.882766, 9.219384]
}

df_rules = pd.DataFrame(data)

# Display the table
print(df_rules.to_string(index=False))

In [ ]:
# Plot Bar Chart for Lift Values
plt.figure(figsize=(10, 6))
plt.barh(df_rules["Antecedents"] + " → " + df_rules["Consequents"], df_rules["Lift"], color='purple')
plt.xlabel("Lift")
plt.ylabel("Association Rule")
plt.title("Top Association Rules by Lift")
plt.gca().invert_yaxis()  # Reverse order for better readability
plt.show()